# Workshop Notebook 4: Automation with ML Workload Orchestrations

Wallaroo provides Data Connections and ML Workload Orchestrations to provide organizations with a method of creating and managing automated tasks that can either be run on demand or a regular schedule.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials Guide: ML Workload Orchestration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/)

## Orchestrations, Taks, and Tasks Runs

We've details how Wallaroo Connections work.  Now we'll use Orchestrations, Tasks, and Task Runs.

| Item | Description |
|---|---|
| Orchestration | ML Workload orchestration allows data scientists and ML Engineers to automate and scale production ML workflows in Wallaroo to ensure a tight feedback loop and continuous tuning of models from training to production. Wallaroo platform users (data scientists or ML Engineers) have the ability to deploy, automate and scale recurring batch production ML workloads that can ingest data from predefined data sources to run inferences in Wallaroo, chain pipelines, and send inference results to predefined destinations to analyze model insights and assess business outcomes. |
| Task | An implementation of an Orchestration.  Tasks can be either `Run Once`:  They run once and upon completion, stop. `Run Scheduled`: The task runs whenever a specific `cron` like schedule is reached.  Scheduled tasks will run until the `kill` command is issued. |
| Task Run | The execusion of a task.  For `Run Once` tasks, there will be only one `Run Task`.  A `Run Scheduled` tasks will have multiple tasks, one for every time the schedule parameter is met.  Task Runs have their own log files that can be examined to track progress and results. |

## Preliminaries

In the blocks below we will preload some required libraries.

For convenience, the following `helper functions` are defined to retrieve previously created workspaces, models, and pipelines:

* `get_workspace(name, client)`: This takes in the name and the Wallaroo client being used in this session, and returns the workspace matching `name`.  If no workspaces are found matching the name, raises a `KeyError` and returns `None`.
* `get_model_version(model_name, workspace)`: Retrieves the most recent model version from the model matching the `model_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.
* `get_pipeline(pipeline_name, workspace)`: Retrieves the most pipeline from the workspace matching the `pipeline_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.

In [1]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

import time
import pyarrow as pa

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
## blank space to log in 

wl = wallaroo.Client()

### Set Configurations

Set the workspace, pipeline, and model used from Notebook 1.  The helper functions will make this task easier.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [3]:
# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-05"

workspace = wl.get_workspace(name=workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

model_name = 'house-price-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'houseprice-estimator'

pipeline = wl.get_pipeline(pipeline_name)

display(workspace)
display(prime_model_version)
display(pipeline)


{'name': 'workshop-workspace-john-05', 'id': 11, 'archived': False, 'created_by': '76b893ff-5c30-4f01-bd9e-9579a20fc4ea', 'created_at': '2024-04-22T20:33:29.332164+00:00', 'models': [{'name': 'house-price-prime', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 47, 17, 816549, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 34, 37, 434083, tzinfo=tzutc())}, {'name': 'house-price-rf-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc())}, {'name': 'house-price-gbr-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc())}], 'pipelines': [{'name': 'houseprice-estimator', 'create_time': datetime.datetime(2024, 4, 22, 20, 34, 49, 55

Name,house-price-prime
Version,be93bbfc-4f0c-444d-aa89-5f15dec01d83
File Name,xgb_model.onnx
SHA,31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-22-Apr 20:47:17


name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-24 16:16:07.840257+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"4d4fdba1-dc5d-4c08-9375-e08ce8738730, b68ae767-dd6b-4f09-a196-a96d2cdaefb1, 3eedd163-bd54-4bde-943e-3f8b32a1f47c, 957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


## Deploy the Pipeline with the Model Version Step

As per the other workshops:

1. Clear the pipeline of all steps.
1. Add the model version as a pipeline step.
1. Deploy the pipeline with the following deployment configuration:

```python
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
```

In [4]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-24 16:18:40.930021+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"eb94fe18-363a-498d-bd36-943289f0f74d, 4d4fdba1-dc5d-4c08-9375-e08ce8738730, b68ae767-dd6b-4f09-a196-a96d2cdaefb1, 3eedd163-bd54-4bde-943e-3f8b32a1f47c, 957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


### Sample Inference

Verify the pipeline is deployed properly with a sample inference with the file `./data/test_data.df.json`.

In [5]:
# sample inference from previous code here

pipeline.infer_from_file('../data/test_data.df.json')

,time,in.tensor,out.variable,anomaly.count
0,2024-04-24 16:19:01.153,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[659806.0],0
1,2024-04-24 16:19:01.153,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[732883.5],0
2,2024-04-24 16:19:01.153,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[419508.84],0
3,2024-04-24 16:19:01.153,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[634028.75],0
4,2024-04-24 16:19:01.153,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[427209.47],0
...,...,...,...,...
3995,2024-04-24 16:19:01.153,"[4.0, 2.25, 2620.0, 98881.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1820.0, 800.0, 47.4662, -122.453, 1728.0, 95832.0, 63.0, 0.0, 0.0]",[436151.13],0
3996,2024-04-24 16:19:01.153,"[3.0, 2.5, 2244.0, 4079.0, 2.0, 0.0, 0.0, 3.0, 7.0, 2244.0, 0.0, 47.2606, -122.254, 2077.0, 4078.0, 3.0, 0.0, 0.0]",[284810.7],0
3997,2024-04-24 16:19:01.153,"[3.0, 1.75, 1490.0, 5000.0, 1.0, 0.0, 1.0, 3.0, 8.0, 1250.0, 240.0, 47.5257, -122.392, 1980.0, 5000.0, 61.0, 0.0, 0.0]",[575571.44],0
3998,2024-04-24 16:19:01.153,"[4.0, 2.5, 2740.0, 5700.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2740.0, 0.0, 47.3535, -122.026, 3010.0, 5281.0, 8.0, 0.0, 0.0]",[432262.38],0


## Sample Orchestration

The orchestration that will automate this process is `./orchestration/real-estate-orchestration.zip`.  The files used are stored in the directory `/orchestration/real-estate-orchestration`, created with the command:

`zip -r real-estate-orchestration.zip real-estate-orchestration/*`.

This contains the following:

* `requirements.txt`:  The Python requirements file to specify the following libraries used.  For this example, that will be empty since we will be using the 
* `main.py`: The entry file that uses a deployed pipeline and performs an inference request against it visible from its log files.
* `data/`: Inference data sources.

The `main.py` script performs a workspace and pipeline retrieval, then an inference against the inference input file.

```python
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas as pd

wl = wallaroo.Client()

# get the arguments
arguments = wl.task_args()

if "workspace_name" in arguments:
    workspace_name = arguments['workspace_name']
else:
    workspace_name="forecast-model-workshop"

if "pipeline_name" in arguments:
    pipeline_name = arguments['pipeline_name']
else:
    pipeline_name="bikedaypipe"

print(f"Workspace: {workspace_name}")
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)
print(workspace)

# the pipeline is assumed to be deployed
print(f"Pipeline: {pipeline_name}")
pipeline = wl.get_pipeline(pipeline_name)
print(pipeline)

print(pipeline.status())

inference_result = pipeline.infer_file_file('./data/test_data.df.json')
print(inference_result)

```

A few things to go over here.  You'll notice this is almost the exact procedures we've been following so far:  we get a workspace and pipeline, pull data from a CSV file, and perform an inference off the data.

This script assumes that the pipeline has already been deployed, and also includes this part:

`arguments = wl.task_args()`

This allows us to pass arguments into a Task created from an Orchestration, so we can specify a different workspace, pipeline, or any other arguments we construct.  This allows orchestrations to be very flexible.

Also, notice that it refers to a specific file:

`inference_result = pipeline.infer_file_file('./data/test_data.df.json')`

In the `forecast-orchestration` directory is the `data` directory with our sample CSV file.  Orchestrations can include additional artifacts.  We could have used a Wallaroo Connection instead, and we encourage you to try that if you want.

## Upload Orchestration

Orchestrations are uploaded with the Wallaroo client `upload_orchestration(path)` method with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **path** | string (Required) | The path to the ZIP file to be uploaded. |

Once uploaded, the deployment will be prepared and any requirements will be downloaded and installed.  A typical orchestration upload looks like this:

```python
my_orchestration = wl.upload_orchestration(path-to-zip-file)
```

### Upload Orchestration Exercise

Try uploading our orchestration from `./forecast-orchestration/forecast-orchestration.zip` - or make your own and upload it.

Once uploaded, you can check the status with the `status()`.  If using the orchestration example above, that would be `my_orchestration.status()`  This is handy to make into a loop to check the status until is shows `ready`.

Here's an example of uploading the Orchestration file, then a loop that will keep checking the status every 5 seconds until it returns `ready`.

```python
orchestration = wl.upload_orchestration(name="my real example", path="./orchestration/real-estate-orchestration.zip")

while orchestration.status() != 'ready':
    print(orchestration.status())
    time.sleep(5)
```


In [6]:
orchestration = wl.upload_orchestration(name="real estate 05 jch sample", path="../orchestration/real-estate-orchestration.zip")

while orchestration.status() != 'ready':
    print(orchestration.status())
    time.sleep(5)

pending_packaging
pending_packaging
pending_packaging
packaging
packaging
packaging
packaging
packaging
packaging
packaging
packaging


## List Orchestrations

Orchestrations are listed with the Wallaroo Client `list_orchestrations()` method.  Orchestrations can be retrieved to a variable by allocated their position in the array - for example:  `orchestration = wl.list_orchestrations()[0]` would return the first orchestration on the list.

### List Orchestrations Exercise

List all of the orchestrations in your Wallaroo instance.  For example, if your client is saved to `wl`, here's some code that would work.

```python
wl.list_orchestrations()
```

In [7]:
# list orchestration here

wl.list_orchestrations()

id,name,status,filename,sha,created at,updated at
a2fe2fa5-de35-43eb-8b0a-d4cfb6c2213b,real estate 05 jch sample,ready,real-estate-orchestration.zip,ec5020...2b7cbe,2024-24-Apr 16:21:50,2024-24-Apr 16:22:50


## Retrieve Orchestration from List

The command `wallaroo.client.list_orchestrations()` returns a List of orchestrations.  We can assign any of the orchestrations in the list to a variable, then use that for other commands.

## Retrieve Orchestration from List Exercise

Use the `list_orchestrations` command and store the orchestration we just uploaded.

Here's some sample code to get you started that stores the last orchestration in the list to the variable `orchestration_from_list`.

```python
orchestration_from_list = wl.list_orchestrations()[-1]
```

In [8]:
# retrieve the orchestration from the list

orchestration_from_list = wl.list_orchestrations()[-1]
orchestration_from_list

Field,Value
ID,a2fe2fa5-de35-43eb-8b0a-d4cfb6c2213b
Name,real estate 05 jch sample
File Name,real-estate-orchestration.zip
SHA,ec502030ee1f50f3816c67ca6b05e26fea720364e27e46aa7626b2628e2b7cbe
Status,ready
Created At,2024-24-Apr 16:21:50
Updated At,2024-24-Apr 16:22:50


## Create Run Once Task from Orchestration

The orchestration is now ready to be implemented as a Wallaroo Task.  We'll just run it once as an example.  This specific Orchestration that creates the Task assumes that the pipeline is deployed, and accepts the arguments:

* workspace_name
* pipeline_name

Tasks are either Run Once, or Run Scheduled.  We create a new task from the Orchestration with either `run_once(task_name, json_args, timeout)` or with `run_scheduled(name, timeout,schedule,json_args)`.  The schedule is based on the Kubernetes cron scheduler.  For example:

```python
schedule={'42 * * * *'}
```

Runs every 42 minutes and contains the answer to life, the universe, and everything.

Creating a scheduled task might be:

```python
task_scheduled = orchestration.run_scheduled(name="schedule example", 
                                             timeout=600, 
                                             schedule=schedule, 
                                             json_args={"workspace_name": workspace_name, 
                                                        "pipeline_name": pipeline_name})
```


### Create Run Once Task from Orchestration Exercise

Using the uploaded orchestration, create a Run Once task using your workspace and pipeline names as the `json_args`.  Here's an example using the variables set above.

```python
task = orchestration.run_once(name="real estate task", 
                              json_args={"workspace_name":workspace_name,
                                         "pipeline_name":pipeline_name}
                              )
```

In [9]:
# create your task here

task = orchestration.run_once(name="real estate task", 
                              json_args={"workspace_name":workspace_name,
                                         "pipeline_name":pipeline_name}
                              )

## Monitor Task Run with Task Status

The Task is the **schedule** to execute the instructions within the orchestration.  The actual execution of the task is the **task run**.  A Run Once task will create one Task Run, while a Run Scheduled task will generate a new Task Run each time the schedule pattern is set.

The status task is viewed with the task `status()` command, where it is either `pending` (no tasks runs are generated yet), or `started` (a task run has been started).

## Monitor Task Run with Task Status Example

We'll monitor the run first with it's status with the `Task.status()` command.

Get the status of the task, and once it is `started` proceed to the next step.  Try doing it as a `while` loop if you feel confident.  Here's some sample code where the task was saved to the variable `task`.

```python
task.status
```

Or as a loop pausing ever 5 seconds until the task status is `started`.

```python
while task.status() != "started":
    display(task.status())
    time.sleep(5)
```


In [10]:
while task.status() != "started":
    display(task.status())
    time.sleep(5)

'pending'

'pending'

'pending'

## List Tasks

The Wallaroo client `list_tasks` method returns a list of tasks, and shows the the last task run status.

### List Tasks Exercise

List the tasks in your Wallaroo instance.  For example, if your Wallaroo client is stored as `wl`, this would show your tasks.

```python
wl.list_tasks()
```

In [11]:
# empty space to list tasks

wl.list_tasks()

id,name,last run status,type,active,schedule,created at,updated at
1ed86029-bf49-4750-8711-551ad9e9a0f1,real estate task,success,Temporary Run,True,-,2024-24-Apr 16:23:29,2024-24-Apr 16:23:44


## Display Task Run Results

The Task Run is the implementation of the task - the actual running of the script and it's results.  Tasks that are Run Once will only have one Task Run, while a Task set to Run Scheduled will have a Task Run for each time the task is executed.  Each Task Run has its own set of logs and results that are monitored through the Task Run `logs()` method.

First, get the Task Run - this is the actual execution of a Task.  The Task is the **scheduled** run of an Orchestration.  The Task Run is the **implementation** of a scheduled Task.  A Run Once Task while generate one Task Run, while a Scheduled Task generated a new Task Run every time the schedule pattern is met until the Task is killed.

We retrieve the task runs with the Task `last_runs()` method, and assign a single Task Run to a variable by selecting it with the list with `last_runs()[index]`.  If you only have one Task Run from a Task, then you can just set the `index` to 0.

### Display Task Run Results Exercise

Retrieve the task run for our generated task, then start checking the logs for our task run.  It may take longer than 30 seconds to launch the task, so be prepared to the command multiple times until is it displayed.  Store the task into a variable for later use.

Here's a code sample where the task was saved to the variable `task`.

```python
task_run = task.last_runs()[0]
task_run
```



In [12]:
task_run = task.last_runs()[0]
task_run

Field,Value
Task,1ed86029-bf49-4750-8711-551ad9e9a0f1
Pod ID,c605ca6d-3b54-45fd-8280-ea87166806ea
Status,success
Created At,2024-24-Apr 16:23:42
Updated At,2024-24-Apr 16:23:42


The Task Run Status is checked with the `_status` method.  This lets you know if there was a failure or if it ran successfully.  If it didn't, you can still get the task run logs to find out why.

In [13]:
task_run._status

'success'

## Retrieve Task Run Logs

The Task Run logs are retrieved with the Wallaroo task runs `log()`, and shows the outputs of the results.  This is why it's useful to have `print` commands in your code to track what it's doing.

### Retrieve Task Run Logs Exercise

Take the task run and display the logs.  It may take a few minutes for the logs to show up, so you may need to refresh the code below a few times.  Here's a quick example of some code.

```python
task_run.logs()
```

In [14]:
task_run.logs()

['2024-04-24T16:23:50.577462639Z stdout F Workspace: workshop-workspace-john-05',
 '2024-04-24T16:23:50.577561999Z stdout F {\'name\': \'workshop-workspace-john-05\', \'id\': 11, \'archived\': False, \'created_by\': \'76b893ff-5c30-4f01-bd9e-9579a20fc4ea\', \'created_at\': \'2024-04-22T20:33:29.332164+00:00\', \'models\': [{\'name\': \'house-price-prime\', \'versions\': 2, \'owner_id\': \'""\', \'last_update_time\': datetime.datetime(2024, 4, 22, 20, 47, 17, 816549, tzinfo=tzutc()), \'created_at\': datetime.datetime(2024, 4, 22, 20, 34, 37, 434083, tzinfo=tzutc())}, {\'name\': \'house-price-rf-model\', \'versions\': 1, \'owner_id\': \'""\', \'last_update_time\': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc()), \'created_at\': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc())}, {\'name\': \'house-price-gbr-model\', \'versions\': 1, \'owner_id\': \'""\', \'last_update_time\': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc()), \'created_at\': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc())}], \'pipelines\': [{\'name\': \'houseprice-estimator\', \'create_time\': datetime.datetime(2024, 4, 22, 20, 34, 49, 559433, tzinfo=tzutc()), \'definition\': \'[]\'}]}',
 "2024-04-24T16:23:50.57758033Z stdout F {'name': 'houseprice-estimator', 'create_time': datetime.datetime(2024, 4, 22, 20, 34, 49, 559433, tzinfo=tzutc()), 'definition': '[]'}",
 '2024-04-24T16:23:50.57757429Z stdout F Pipeline: houseprice-estimator',
 "2024-04-24T16:23:50.577586719Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.3.107', 'name': 'engine-56f9586b8f-sddd5', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'houseprice-estimator', 'status': 'Running', 'version': 'eb94fe18-363a-498d-bd36-943289f0f74d'}]}, 'model_statuses': {'models': [{'name': 'house-price-prime', 'sha': '31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c', 'status': 'Running', 'version': 'be93bbfc-4f0c-444d-aa89-5f15dec01d83'}]}}], 'engine_lbs': [{'ip': '10.28.2.210', 'name': 'engine-lb-d7cc8fc9c-j9ncn', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-04-24T16:23:50.577591159Z stdout F                         time  ... anomaly.count',
 '2024-04-24T16:23:50.57759541Z stdout F 0    2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.57760203Z stdout F 1    2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.577616359Z stdout F 4    2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.57760611Z stdout F 2    2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.577612159Z stdout F 3    2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.57762451Z stdout F 3995 2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.57762081Z stdout F ...                      ...  ...           ...',
 '2024-04-24T16:23:50.577628179Z stdout F 3996 2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.577636039Z stdout F 3997 2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.57764919Z stdout F 3999 2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.577641799Z stdout F 3998 2024-04-24 16:23:50.368  ...             0',
 '2024-04-24T16:23:50.57765873Z stdout F [4000 rows x 4 columns]',
 '2024-04-24T16:23:50.577654499Z stdout F ']

You have now walked through setting up a basic assay and running it over historical data.

## Congratulations!
In this workshop you have
* Deployed a single step house price prediction pipeline and sent data to it.
* Uploaded an ML Orchestration into Wallaroo.
* Created a Run Once Task from the Orchestration.
* Viewed the Task Run's status generated from the Task.
* Viewed the Task Run's logs.

Great job! 

### Cleaning up.

Now that the workshop is complete, don't forget to undeploy your pipeline to free up the resources.

In [15]:
pipeline.undeploy()

name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-24 16:18:40.930021+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"eb94fe18-363a-498d-bd36-943289f0f74d, 4d4fdba1-dc5d-4c08-9375-e08ce8738730, b68ae767-dd6b-4f09-a196-a96d2cdaefb1, 3eedd163-bd54-4bde-943e-3f8b32a1f47c, 957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False
